In [1]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute]
                   [--source [SOURCE]] [--quiet] [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US

In [24]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyACM1, /dev/ttyS0, /dev/ttyS1, /dev/ttyS2, /dev/ttyS3 
Connecting to --port=/dev/ttyACM1 --baud=115200 
Ready.


In [ ]:
%ls --recurse

Listing directory '/'.
      139    boot.py
       80    config.txt
             mqtt_as/
     2549    stdmqttas.py

mqtt_as:
    36258    mqtt_as/__init__.py
     1897    mqtt_as/async_message.py
     2029    mqtt_as/clean.py
    36258    mqtt_as/junk.txt
     2377    mqtt_as/lptest_min.py
       41    mqtt_as/main.py
     2173    mqtt_as/mqtt_as_timeout.py
     7385    mqtt_as/mqtt_v5_properties.py
     2504    mqtt_as/range.py
     3413    mqtt_as/range_ex.py
     2538    mqtt_as/tls.py
     1917    mqtt_as/tls32.py
     2445    mqtt_as/tls8266.py
     2148    mqtt_as/unclean.py
             mqtt_as/v5/

mqtt_as/v5:


>>

In [19]:
import os
os.mkdir("mqtt_as")
%sendtofile --source stdmqttas.py
# transferring of directories is buggy.  doesn't like / symbols here either

Sent 75 lines (2549 bytes) to stdmqttas.py.


In [ ]:

# Simplify and deploy on dot matrix at front
# check watchdog does or doesn't work
# report mdns not working on esp8266 (and the watchdog)
# check if timeout on wifi allows alternate wifi passwords to be given

# Make sure the firmware is
# %esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

In [ ]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# clk red      D5
# cs  orange   D8
# din yellow   D7
# gnd green    G
# vcc blue     5V

In [138]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyUSB0, /dev/ttyS0, /dev/ttyS1, /dev/ttyS2, /dev/ttyS3 
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [139]:
%sendtofile config.txt
wifiname  DoESLiverpool
wifipassword decafbad00
mqttbroker  10.0.100.1
pinled       -2
pinsck       14
pincs        15
pinmosi      13
numledchars  20

Sent 8 lines (150 bytes) to config.txt.


In [137]:
%sendtofile --source wifimqttconnections.txt config.txt
%sendtofile config.txt
wifiname  DoESLiverpool,decafbad00,mosquitto.doesliverpool.xyz
pinled       -2
pinsck       21
pincs        17
pinmosi      16
numledchars  20

Sent 8 lines (150 bytes) to config.txt.


In [134]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyUSB0, /dev/ttyS0, /dev/ttyS1, /dev/ttyS2, /dev/ttyS3 
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [135]:
%sendtofile --source stdmqttas_esp8266.py stdmqttas.py


Sent 99 lines (3341 bytes) to stdmqttas.py.


In [82]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 102 lines (3359 bytes) to stdmqttas.py.
Sent 39 lines (1151 bytes) to utils.py.


In [79]:
%sendtofile ledstrippanel.py

import framebuf
from machine import Pin, SPI

cs, spi = None, None
brightnesschars = None
buffer = None
fbuff = None
numledchars = 0
# ESP32-HSPI: sck=14, mosi=13

def setbrightness(brightness, i0=0, i1=9999):
    # 0<=brightness<=15
    for i in range(max(i0, 0), min(i1, numledchars)):
        brightnesschars[(numledchars - i)*2 - 1] = brightness
        
def showbrightness():
    cs.value(0)  
    spi.write(brightnesschars)
    cs.value(1)
    
def scrollbrightness(brightness=0):
    for i in range(numledchars - 1):
        brightnesschars[(numledchars - i)*2 - 1] = brightnesschars[(numledchars - (i+1))*2 - 1]
    brightnesschars[1] = brightness
    
s = bytearray(8)
def show():
    for y in range(8):
        s[6] = s[4] = s[2] = s[0] = y+1
        yp = y*(numledchars) + numledchars
        cs.value(0)  # cannot put in loop as resets the shifting
        for m in range(0, numledchars-1, 4):
            s[1] = buffer[yp - m - 1]
            s[3] = buffer[yp - m - 2]
            s[5] = buffer[yp - m - 3]
            s[7] = buffer[yp - m - 4]
            spi.write(s)
        cs.value(1)

def init():
    for i in range(0, 10, 2):
        cs.value(0)
        x = b"\x0c\x00\x0f\x00\x0b\x07\x09\x00\x0c\x01"[i:i+2]
        spi.write(x*numledchars)
        cs.value(1)
    showbrightness()
        
def setup(lnumledchars, lcs, lspi):
    global numledchars, cs, spi, brightnesschars, buffer, fbuff
    cs, spi = lcs, lspi
    numledchars = lnumledchars # can be 4n+1 for scroll buffer

    brightnesschars = bytearray(bytes([0x0a, 0])*numledchars)
    buffer = bytearray(8*(numledchars))  # make one extra character to scroll from
    fbuffwidth = 8*(numledchars)
    fbuffheight = 8
    fbuff = framebuf.FrameBuffer(buffer, fbuffwidth, fbuffheight, framebuf.MONO_HLSB)

    init()
    
    for i in range(0, numledchars*8, 4):
        fbuff.fill_rect(i, (i%8), 4, 4, 1)
    show()
    return fbuff


Sent 64 lines (1913 bytes) to ledstrippanel.py.


In [140]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyUSB1, /dev/ttyS0, /dev/ttyS1, /dev/ttyS2, /dev/ttyS3 
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [141]:
#%sendtofile --source stdmqttas.py
%sendtofile --source stdmqttas_esp8266.py stdmqttas.py

Sent 90 lines (2926 bytes) to stdmqttas.py.


In [88]:
import stdmqttas


In [89]:
print(stdmqttas.pinled)

Pin(2)


In [125]:
from machine import Pin, PWM, SPI
spi = SPI(1, 1000000, sck=int(fconfig["pinsck"]), mosi=int(fconfig["pinmosi"]))
print(spi)
help(spi)

SPI(id=1, baudrate=1000000, polarity=0, phase=0, bits=8, firstbit=0, sck=21, mosi=16, miso=12)
object SPI(id=1, baudrate=1000000, polarity=0, phase=0, bits=8, firstbit=0, sck=21, mosi=16, miso=12) is of type SPI
  init -- <function>
  deinit -- <function>
  read -- <function>
  readinto -- <function>
  write -- <function>
  write_readinto -- <function>
  MSB -- 0
  LSB -- 1


In [126]:
import ledstrippanel
from machine import Pin, PWM, SPI
fbuff = setup(numledchars, Pin(int(fconfig["pincs"]), Pin.OUT), SPI(1, 1000000))


Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
NameError: name 'setup' isn't defined


In [114]:
ledstrippanel.fbuff.fill(0)
ledstrippanel.show()

In [133]:
%sendtofile main.py

from mqtt_as import MQTTClient
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask, sclient
import uasyncio as asyncio
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count
from machine import Pin, PWM, SPI
import time
from ledstrippanel import setup, show
#from uasyncio.queues import Queue

numledchars = int(fconfig["numledchars"])
spi = SPI(1, 1000000, sck=int(fconfig["pinsck"]), mosi=int(fconfig["pinmosi"]))
fbuff = setup(numledchars, Pin(int(fconfig["pincs"]), Pin.OUT), spi)
flashpinled(5, 300, 300)

topicmessage = topicstem+"/message"
topiccmd = topicstem+"/cmd"
topicstatus = topicstem+"/status"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"

#qmessages = Queue()
qmessages = [ ]
async def messagetask():
    message = " "
    while True:
        if len(qmessages) == 0:
            await asyncio.sleep_ms(100)
            continue
        #prevmessage, message = message, await(qmessages.get())
        prevmessage, message = message, qmessages.pop(0)
        for xchar, (c1, c2) in enumerate(zip(prevmessage, message)):
            if c1 != c2:
                break
        fbuff.fill(0)
        fbuff.text(message, 0, 0, 1)
        for ys in range(7, -1, -1):
            fbuff.fill_rect(xchar*8, 0, (numledchars - xchar)*8, 8, 0)
            fbuff.text(prevmessage[xchar:], xchar*8, ys - 8, 1)
            fbuff.text(message[xchar:], xchar*8, ys, 1)
            show()
            await asyncio.sleep_ms(50)
        await asyncio.sleep_ms(450)

async def cursorflash():
    for i in itertools_count():
        await asyncio.sleep_ms(1000)
        if not sclient():
            continue
        client = sclient()
        if client.connectioncount and not client.isconnected() and len(qmessages) < 2:
            qmessages.append("wifi fail: %d"%i)
            await asyncio.sleep_ms(5000)
            ipnumber = client._sta_if.ifconfig()[0]
            qmessages.append(ipnumber)
            await asyncio.sleep_ms(4000)
        elif len(qmessages) == 0:
            fbuff.fill_rect((numledchars-1)*8, 7, 8, 1, i%2)
            show()
        if client.isconnected():
            await client.publish(topicstatus, "beat-%d"%i)
        
def callbackcmd(topic, msg, retained):
    print("callbackcmd", topic, msg)
    if topic == topiccmd:
        aloop.create_task(callbackcmdtask(clientsingleton[0], topicreply, msg))
    elif len(qmessages) < 5:
        if 1 <= len(msg) <= 50:
            #aloop.create_task(qmessages.put(msg.decode()))
            qmessages.append(msg.decode())

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    qmessages.append("subscribing")
    await client.subscribe(topiccmd)
    print("subscribed ", topiccmd)
    await client.subscribe(topicmessage)
    print("subscribed ", topicmessage)
    qmessages.append("ip: "+ipnumber)
    await client.publish(topicmessage, "reconnections #%d"%client.connectioncount)
    client.connectioncount += 1
    print("subscribed")
            
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
#client = MQTTClient(config)
#client.DEBUG = True
#client.connectioncount = 0

aloop = asyncio.get_event_loop()
aloop.create_task(messagetask())
aloop.create_task(cursorflash())
aloop.create_task(mqttconnecttask())
aloop.create_task(flashledconnectedtask())
aloop.run_forever()


Sent 96 lines (3480 bytes) to main.py.


In [18]:
fbuff.fill(0)
show()

In [9]:
help("modules")


__main__          collections       machine           ssl
_asyncio          cryptolib         math              struct
_boot             deflate           micropython       sys
_espnow           dht               mip/__init__      time
_onewire          ds18x20           neopixel          tls
_webrepl          errno             network           uasyncio
apa102            esp               ntptime           uctypes
array             espnow            onewire           umqtt/robust
asyncio/__init__  flashbdev         os                umqtt/simple
asyncio/core      framebuf          platform          upysh
asyncio/event     gc                port_diag         urequests
asyncio/funcs     hashlib           random            vfs
asyncio/lock      heapq             re                webrepl
asyncio/stream    inisetup          requests/__init__ webrepl_setup
binascii          io                select            websocket
btree             json              socket
builtins          lwip      